## Inicializar la Dimensión Cliente
Importaciones

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import yaml

# Añadir conexión a la base de datos

In [2]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']

# Construir la URL de conexión para cada base de datos
url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@"
           f"{config_ryf['host']}:{config_ryf['port']}/{config_ryf['dbname']}")

url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@"
           f"{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}")

# Crear el Engine de SQLAlchemy para ambas conexiones
ryf_conn = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

# Realizar la extracción de datos

In [3]:
# Cargar la tabla cliente
df_cliente = pd.read_sql('SELECT * FROM public.cliente', ryf_conn)
# Cargar la tabla ciudad
df_ciudad = pd.read_sql('SELECT ciudad_id, nombre AS ciudad, departamento_id FROM public.ciudad', ryf_conn)
# Cargar la tabla departamento
df_departamento = pd.read_sql('SELECT departamento_id, nombre AS departamento FROM public.departamento', ryf_conn)
# Cargar la tabla tipo_cliente
df_tipo_cliente = pd.read_sql('SELECT tipo_cliente_id, nombre AS tipo_cliente FROM public.tipo_cliente', ryf_conn)


# Realizar la transformación
 Realiza las uniones entre las tablas para enriquecer la información de la dimensión Cliente

In [4]:
# Unir cliente con ciudad
df_cliente_ciudad = pd.merge(df_cliente, df_ciudad, on='ciudad_id', how='left')

# Unir el resultado con departamento
df_cliente_ciudad_departamento = pd.merge(df_cliente_ciudad, df_departamento, on='departamento_id', how='left')

# Unir con tipo_cliente para obtener el nombre del tipo de cliente
df_dim_cliente = pd.merge(df_cliente_ciudad_departamento, df_tipo_cliente, on='tipo_cliente_id', how='left')

# Seleccionar las columnas relevantes para la dimensión cliente
df_dim_cliente = df_dim_cliente[[
    'cliente_id', 'nit_cliente', 'nombre', 'email', 'direccion', 'telefono',
    'nombre_contacto', 'ciudad', 'departamento', 'tipo_cliente', 'activo', 'sector'
]]


# Visualizar la dimensión Cliente

In [5]:
df_dim_cliente.head()

,cliente_id,nit_cliente,nombre,email,direccion,telefono,nombre_contacto,ciudad,departamento,tipo_cliente,activo,sector
0,1,25,Cliente 2,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,CALI,VALLE DEL CAUCA,Persona Juridica,True,S
1,2,123,Cliente 1,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,CALI,VALLE DEL CAUCA,Persona Juridica,True,industrial
2,6,24390-3,CLINICA DEPORTIVA DEL SUR,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,CALI,VALLE DEL CAUCA,Persona Juridica,True,salud
3,19,8301821,HOSPITAL ORTOPEDICO DE COLOMBIA,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,CALI,VALLE DEL CAUCA,Persona Juridica,True,salud
4,8,5017350-8,CLINICA NEFROLOGOS DE CALI,algo.com,Calle 100 No 25-18,327-00000,Cristiano Ronaldo,CALI,VALLE DEL CAUCA,Persona Juridica,True,salud


# Cargar la dimensión Cliente al ETL

In [6]:
df_dim_cliente.to_sql(
    'dim_cliente',  # Nombre de la tabla destino en la base de datos.
    con=etl_conn,   # Conexión a la base de datos ETL.
    if_exists='replace',  # Reemplaza la tabla si ya existe.
    index_label='key_dim_cliente'  # Nombre de la columna de índice (clave primaria).
)

27